In [28]:
import sklearn
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import concurrent
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [3]:
X, y = load_digits(return_X_y = True)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [36]:
y_train[[i for i in range(1347)]]

array([1, 7, 2, ..., 0, 3, 9])

In [74]:
def return_bootstrap(X_train, y_train):
    len_ = X_train.shape[0]
    indices = np.array([i for i in range(len_)])
    indice = np.random.choice(indices, size = len(indices),replace = True).reshape(-1)
    return X_train[indice], y_train[indice], indice
X_train_b, y_train_b, indice = return_bootstrap(X_train, y_train)


/home/sahmaran/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.9888888888888889

In [75]:
L = []
for i in range(1000):
    X_train_b, y_train_b, indice = return_bootstrap(X_train, y_train)
    knn.fit(X_train_b, y_train_b)
    L.append(knn.score(X_test, y_test))

/home/sahmaran/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/sahmaran/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be ac

In [77]:
np.mean(L)

0.981528888888889

In [81]:
R = []
knn = KNeighborsClassifier(n_neighbors= 10)
for i in range(1000):
    X_train_b, y_train_b, indice = return_bootstrap(X_train, y_train)
    knn.fit(X_train_b, y_train_b)
    R.append(knn.score(X_test, y_test))

/home/sahmaran/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/sahmaran/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be ac

In [83]:
np.mean(R)

0.9754688888888889